In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()

In [4]:
X = df.drop('label', axis = 1)

In [5]:
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
import tensorflow as tf
from keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM, Dense

2023-10-23 20:28:44.175840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 20:28:45.948541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/harsh/miniconda3/envs/tf/lib/:/home/harsh/miniconda3/envs/tf/lib/
2023-10-23 20:28:45.952709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/harsh/miniconda3/envs/tf/lib/:/home/harsh/minicond

In [8]:
voc_size = 5000

In [9]:
messages = X.copy()

In [10]:
messages.reset_index(inplace=True)

In [11]:
import nltk
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/harsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4021, 4011, 3762, 3840, 2635, 1034, 2056, 2804, 3433, 1531],
 [3610, 4142, 2486, 1205, 293, 3202, 2578],
 [2407, 477, 868, 4217],
 [3105, 989, 1569, 4997, 1633, 2488],
 [2904, 293, 3200, 3826, 3372, 1439, 293, 1952, 2785, 198],
 [4327,
  3138,
  1113,
  456,
  213,
  626,
  2181,
  3850,
  3784,
  1563,
  2253,
  2073,
  222,
  2134,
  2578],
 [1062, 4854, 2694, 387, 235, 649, 1419, 4391, 2174, 4721, 3320],
 [857, 3732, 1707, 793, 748, 2828, 626, 3058, 2174, 4721, 3320],
 [4504, 1444, 3806, 4517, 769, 4588, 1017, 1231, 626, 2701],
 [3123, 901, 847, 3576, 2354, 4657, 3810, 608],
 [4304, 912, 3772, 3164, 245, 1395, 171, 3441, 4061, 2190, 33],
 [4997, 1488, 2635, 4588, 626, 748],
 [2054, 3210, 3657, 2585, 1940, 3940, 4402, 2557, 703],
 [2361, 1753, 543, 1824, 2832, 4909, 1220, 2174, 4721, 3320],
 [4039, 573, 4848, 3198, 856, 2174, 4721, 3320],
 [3552, 322, 1426, 4714, 4791, 2783, 609, 347, 4886, 4381],
 [1070, 1066, 4142],
 [1105, 123, 2763, 3370, 626, 863, 2251, 2578],
 [201, 4995, 248

In [15]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2804 3433 1531]
 [   0    0    0 ...  293 3202 2578]
 [   0    0    0 ...  477  868 4217]
 ...
 [   0    0    0 ... 2174 4721 3320]
 [   0    0    0 ... 2718 1653 2475]
 [   0    0    0 ... 3524 3477  418]]


In [16]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4021,
       4011, 3762, 3840, 2635, 1034, 2056, 2804, 3433, 1531], dtype=int32)

In [17]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


2023-10-23 20:29:04.653153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:29:04.727078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:29:04.727273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 20:29:04.730533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
len(embedded_docs),y.shape


(18285, (18285,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 4s 11ms/step - loss: 9.6103e-04 - accuracy: 0.9996 - val_loss: 0.7114 - val_accuracy: 0.9051
Epoch 2/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0052 - accuracy: 0.9980 - val_loss: 0.6326 - val_accuracy: 0.9019
Epoch 3/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0079 - accuracy: 0.9969 - val_loss: 0.5713 - val_accuracy: 0.9137
Epoch 4/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.5353 - val_accuracy: 0.9036
Epoch 5/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.6275 - val_accuracy: 0.9109
Epoch 6/10
383/383 [==============================] - 4s 10ms/step - loss: 6.8794e-04 - accuracy: 0.9998 - val_loss: 0.7054 - val_accuracy: 0.9105
Epoch 7/10
383/383 [==============================] - 4s 10ms/step - loss: 2.5797e-04 - accuracy: 0.9999 - val_loss: 0.7357 - val_accu